<a href="https://colab.research.google.com/github/alvictal/stockscope/blob/main/StockScope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U google-generativeai

In [2]:
!pip install -q -U yfinance


In [ ]:
import yfinance as yf
import google.generativeai as genai
import textwrap
from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata

def to_markdown(text):
  text = text.replace('$', '\$')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


# Armazenamento da chave de API
api_key = userdata.get("SKEY")

# Configuração da chave de API
genai.configure(api_key = api_key)

generation_config = {
    "candidate_count" : 1,
    "temperature" : 0.8,
    "top_p": 0.90,
    "top_k" : 32
}

safety_settings = {
    "HARASSMENT" : "BLOCK_MEDIUM_AND_ABOVE",
    "HATE" : "BLOCK_MEDIUM_AND_ABOVE",
    "SEXUAL" : "BLOCK_MEDIUM_AND_ABOVE",
    "DANGEROUS" : "BLOCK_MEDIUM_AND_ABOVE"
}

model = genai.GenerativeModel("gemini-1.0-pro",
                              generation_config = generation_config,
                              safety_settings = safety_settings)

#Input do usuário para qual ação deseja olhar
stock = input("Qual ação você deseja fazer uma analise (exemplo PETR4.SA): ")
stock = stock.upper()
ticket = yf.Ticker(stock)

#prompts de analise para informação de fluxo de caixa
prompt = f'Como uma pessoa com pouco conhecimento em ações \
Faça uma analise da ação {stock} baseado nas informações de fluxo de caixa da empresa. \
As informações the fluxo de caixa  estão \
tabuladas abaixo\n'
prompt += ticket.quarterly_cashflow.to_markdown()

#guardando informação de fluxo de caixa para uso no ultimo prompt de conclusão
response = model.generate_content(prompt)
response_text = "# Analise de Fluxo de caixa \n"
response_text += response.text
response_text += "\n\n"

#prompt de analise para informação de balanço financeiro
prompt = f'Como uma pessoa com pouco conhecimento em ações \
Faça uma analise da ação {stock} baseado nas informações de balanço da empresa. \
As informações balanço estão \
tabuladas abaixo\n'
prompt += ticket.quarterly_balancesheet.to_markdown()

#guardando informação de balanço financeiro para uso no ultimo prompt de conclusão
response = model.generate_content(prompt)
response_text += "# Analise do balanço financeiro da empresa \n"
response_text += response.text
response_text += "\n\n"

#Tirando conclusão com as informações obtidas
prompt = "Através das informações abaixo, conclua se vale investir na empresa, indicando pontos positivos e pontos negaticos\n"
prompt += response_text


#Imprimindo as informações para o usuário
response = model.generate_content(prompt)
display(to_markdown(response_text))
display(to_markdown("# Conclusão"))
display(to_markdown(response.text))

